#Configurazione Kaggle

In [1]:
from google.colab import files

# Carica il tuo kaggle.json dal PC
files.upload()   # seleziona kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle (9).json


#Download dataset

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip chest-xray-pneumonia.zip


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  chest-xray-pneumonia.zip
replace chest_xray/__MACOSX/._chest_xray? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

#Generatori di immagini

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "chest_xray/train"
val_dir = "chest_xray/val"

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(train_dir, target_size=(150,150), batch_size=32, class_mode="binary")
val_gen = datagen.flow_from_directory(val_dir, target_size=(150,150), batch_size=32, class_mode="binary")


ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-1' coro=<Kernel.poll_control_queue() done, defined at /usr/local/lib/python3.12/dist-packages/ipykernel/kernelbase.py:277> wait_for=<Future finished result=<Future at 0x...state=pending>> cb=[_chain_future.<locals>._call_set_state() at /usr/lib/python3.12/asyncio/futures.py:396]>


#Modello CNN e training

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(train_gen, epochs=5, validation_data=val_gen)

# Salva il modello
model.save("modello_pneumonia.h5")


#Upload immagine radiografica dal PC

In [ ]:
from google.colab import files
uploaded = files.upload()   # seleziona la tua radiografia .jpg

img_path = list(uploaded.keys())[0]   # prende il nome del file caricato


#Predizione sul modello

In [ ]:
import numpy as np

model = tf.keras.models.load_model("modello_pneumonia.h5")

img = tf.keras.preprocessing.image.load_img(img_path, target_size=(150, 150))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

preds = model.predict(img_array)
print("Predizione:", preds)


#Grad‑CAM

In [ ]:
import matplotlib.pyplot as plt
import cv2

# Controlla il nome del layer convoluzionale
model.summary()

# Usa l’ultimo Conv2D (es. "conv2d_2")
grad_model = tf.keras.models.Model(
    inputs=model.input,
    outputs=[model.get_layer("conv2d_2").output, model.output]
)

with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(img_array)
    loss = predictions[:, np.argmax(predictions[0])]

grads = tape.gradient(loss, conv_outputs)
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

conv_outputs = conv_outputs[0]
heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
heatmap = tf.squeeze(heatmap)
heatmap = np.maximum(heatmap, 0) / np.max(heatmap)

# Sovrapponi la heatmap
img_cv = cv2.imread(img_path)
heatmap = cv2.resize(heatmap.numpy(), (img_cv.shape[1], img_cv.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = heatmap * 0.4 + img_cv

plt.imshow(cv2.cvtColor(superimposed_img.astype("uint8"), cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()
